## 移除最后conv最后两层网络


In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [6]:
batch_size = 256
num_epochs = 200
extract_size = 512
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderCut2Layer()
encoder = encoder.to(device)

# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
# data_loader.dataset.save_to("./bottle_neck/resnet152_train_remove_last_2_conv.h")
data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train_remove_last_2_conv.h')

encoding 32738/32739->100.00%, spent_time:18:48.09

### init model

使用两层网络,结果过拟合,训练准确率很高,验证结果比较差,使用再试

In [9]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = FinalClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### -> Load the best trained model,yet!

In [21]:
classify_model.load_state_dict(torch.load('./models/class_single_rm2layer_last.pkl'))

### time to train model

In [16]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [15]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(301, num_epochs+301):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_rm2layer_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_rm2layer_last.pkl'))

Epoch [201/100], Step [120/127], Loss: 2.2355, Accuracy: 49.61%, Best_acc: 53.52%            
 Epoch 201, spent time:5.66s
Epoch [202/100], Step [120/127], Loss: 2.0659, Accuracy: 54.30%, Best_acc: 58.59%            
 Epoch 202, spent time:5.67s
Epoch [203/100], Step [120/127], Loss: 2.1102, Accuracy: 50.00%, Best_acc: 59.77%            
 Epoch 203, spent time:5.64s
Epoch [204/100], Step [120/127], Loss: 2.1021, Accuracy: 50.78%, Best_acc: 59.77%            
 Epoch 204, spent time:5.82s
Epoch [205/100], Step [120/127], Loss: 1.9901, Accuracy: 57.03%, Best_acc: 59.77%            
 Epoch 205, spent time:5.67s
Epoch [206/100], Step [120/127], Loss: 2.0565, Accuracy: 54.30%, Best_acc: 59.77%            
 Epoch 206, spent time:5.65s
Epoch [207/100], Step [120/127], Loss: 2.1219, Accuracy: 49.22%, Best_acc: 59.77%            
 Epoch 207, spent time:5.68s
Epoch [208/100], Step [120/127], Loss: 2.0468, Accuracy: 55.08%, Best_acc: 59.77%            
 Epoch 208, spent time:5.71s
Epoch [209/100],

Epoch [267/100], Step [120/127], Loss: 1.8195, Accuracy: 60.16%, Best_acc: 62.89%            
 Epoch 267, spent time:5.58s
Epoch [268/100], Step [120/127], Loss: 1.9801, Accuracy: 51.56%, Best_acc: 62.89%            
 Epoch 268, spent time:5.66s
Epoch [269/100], Step [120/127], Loss: 1.9060, Accuracy: 56.64%, Best_acc: 62.89%            
 Epoch 269, spent time:5.66s
Epoch [270/100], Step [120/127], Loss: 1.8854, Accuracy: 53.12%, Best_acc: 62.89%            
 Epoch 270, spent time:5.63s
Epoch [271/100], Step [120/127], Loss: 1.9166, Accuracy: 54.69%, Best_acc: 64.06%            
 Epoch 271, spent time:5.64s
Epoch [272/100], Step [120/127], Loss: 1.9655, Accuracy: 53.12%, Best_acc: 64.06%            
 Epoch 272, spent time:5.68s
Epoch [273/100], Step [120/127], Loss: 1.8967, Accuracy: 56.25%, Best_acc: 64.45%            
 Epoch 273, spent time:5.59s
Epoch [274/100], Step [120/127], Loss: 1.9068, Accuracy: 53.12%, Best_acc: 64.45%            
 Epoch 274, spent time:5.73s
Epoch [275/100],

### How good is the model?

In [11]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_rm2layer.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_rm2layer.h")

encoding 4981/4982->99.98%, spent_time:2:56.20

In [29]:
classify_model.load_state_dict(torch.load('./models/class_single_rm2layer_last.pkl'))

In [16]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 52.83%.
